# HW1_TASK4

## Experiment

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import re

we still use the rotten tamatoes dataset for sentiment analysis https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews/data
Here we only use the training data

In [2]:
df = pd.read_csv('/Users/jianwenliu/Documents/IndianaUniversity/courses/nlp/nlphw/train.tsv',sep='\t')
df.head(10)

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
5,6,1,of escapades demonstrating the adage that what...,2
6,7,1,of,2
7,8,1,escapades demonstrating the adage that what is...,2
8,9,1,escapades,2
9,10,1,demonstrating the adage that what is good for ...,2


In [3]:
#limit all sentiment<3 to 0, otherwise enforce it to be 1
df.Sentiment[df['Sentiment']<3]=0
df.Sentiment[df['Sentiment']>=3]=1

df.head(10)

/Users/jianwenliu/anaconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/jianwenliu/anaconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,0
1,2,1,A series of escapades demonstrating the adage ...,0
2,3,1,A series,0
3,4,1,A,0
4,5,1,series,0
5,6,1,of escapades demonstrating the adage that what...,0
6,7,1,of,0
7,8,1,escapades demonstrating the adage that what is...,0
8,9,1,escapades,0
9,10,1,demonstrating the adage that what is good for ...,0


In [4]:
df = df.sample(frac=0.1)

In [5]:
df = df.drop(['PhraseId', 'SentenceId'], axis=1)

In [6]:
df.head()
y_all = df.values[:,-1].reshape(-1,1)

In [7]:
df.shape

(15606, 2)

In [8]:
y_all

array([[1],
       [1],
       [0],
       ...,
       [0],
       [1],
       [0]], dtype=object)

In [9]:
def clean(text):
    return re.compile(u'[^a-zA-Z0-9]').sub(' ',text)   
df['Phrase'] = df['Phrase'].apply(lambda x: clean(x))

## Spacy vector

## Here we use 'en_core_web_md' for Spacy vectors

In [10]:
import spacy

nlp = spacy.load('en_core_web_md')
tokens = nlp(u'dog cat banana afskfsd')



for token in tokens:
    print(token.text, token.has_vector, token.vector_norm, token.is_oov)

dog True 7.0336733 False
cat True 6.6808186 False
banana True 6.700014 False
afskfsd False 0.0 True


 Attention, the vector here is normalized vectors

## prepare for the normalized vector

vectorize on vector_norm

In [11]:
def norm_vec(df):
    comm_vec = []
    for com in df.Phrase.values:
        tokens = nlp(com)
        comm_row = []
        for token in tokens:
            token_vec = token.vector_norm
            comm_row.append(token_vec)
        comm_vec.append(comm_row)
    return comm_vec,nlp

Limit 200 tokens for comment

In [12]:
comm_vec,nlp=norm_vec(df)
comm_vec_tri = [comm[:200] for comm in comm_vec]
import gc
del comm_vec, df, nlp
gc.collect()

7

zero paddings

In [13]:
def zero_pad(A):
    i = 0
    zero_vec = np.zeros(1)
    for comm in A:
        for k in range(0,200-len(comm)):
            comm.insert(0,zero_vec)
        A[i] = comm
        i += 1
    return A

In [14]:
comm_vec_tri=zero_pad(comm_vec_tri)
X = np.array(comm_vec_tri)
import gc
del comm_vec_tri
gc.collect()

0

In [15]:
print(X.shape)
print(y_all.shape)

(15606, 200)
(15606, 1)


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y_all, test_size=0.2)

In [17]:
X_train.shape

(12484, 200)

check if train and test has the same 1d

In [18]:
X_test.shape

(3122, 200)

In [19]:
y_train.shape

(12484, 1)

In [20]:
X_train
#X_train=X_train.astype('float')

array([[array([0.]), array([0.]), array([0.]), ..., 6.30519, 4.97793,
        6.0422654],
       [array([0.]), array([0.]), array([0.]), ..., 6.5976605, 6.0594854,
        4.8260193],
       [array([0.]), array([0.]), array([0.]), ..., 6.9073033, 6.871181,
        0.0],
       ...,
       [array([0.]), array([0.]), array([0.]), ..., array([0.]),
        array([0.]), 5.3285394],
       [array([0.]), array([0.]), array([0.]), ..., 5.1979666, 5.5782003,
        5.49947],
       [array([0.]), array([0.]), array([0.]), ..., array([0.]),
        5.306696, 6.7812724]], dtype=object)

make the y types to int for further classification

In [21]:
y_train
y_train=y_train.astype('int')


In [22]:
y_test
y_test=y_test.astype('int')

## logistic regression

In [23]:
from sklearn.linear_model import LogisticRegression

# train the model
clf = LogisticRegression()
clf.fit(X_train, y_train)

/Users/jianwenliu/anaconda3/envs/nlp/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jianwenliu/anaconda3/envs/nlp/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [24]:
from sklearn.metrics import accuracy_score

## Result

In [25]:
#predics
y_pre = clf.predict(X_test)
acc = accuracy_score(y_test, y_pre)
print("Accuracy is ", acc)

Accuracy is  0.7370275464445868


## Fasttext vector

we use gensim to load the fasttext vector, the 'wiki-news-300d-1M.vec' is on the Fasttext web:
https://fasttext.cc/docs/en/english-vectors.html

In [79]:
from __future__ import print_function
import gensim
print(gensim.__version__)

3.7.1


In [80]:
from gensim.models import KeyedVectors
# Creating the model
en_model = KeyedVectors.load_word2vec_format('wiki-news-300d-1M.vec')

# Getting the tokens 
words = []
for word in en_model.vocab:
    words.append(word)

# Printing out number of tokens available
print("Number of Tokens: {}".format(len(words)))

# Printing out the dimension of a word vector 
print("Dimension of a word vector: {}".format(
    len(en_model[words[0]])
))

# Print out the vector of a word 
print("Vector components of a word: {}".format(
    en_model[words[0]]
))

Number of Tokens: 999994
Dimension of a word vector: 300
Vector components of a word: [ 1.0730e-01  8.9000e-03  6.0000e-04  5.5000e-03 -6.4600e-02 -6.0000e-02
  4.5000e-02 -1.3300e-02 -3.5700e-02  4.3000e-02 -3.5600e-02 -3.2000e-03
  7.3000e-03 -1.0000e-04  2.5800e-02 -1.6600e-02  7.5000e-03  6.8600e-02
  3.9200e-02  7.5300e-02  1.1500e-02 -8.7000e-03  4.2100e-02  2.6500e-02
 -6.0100e-02  2.4200e-01  1.9900e-02 -7.3900e-02 -3.1000e-03 -2.6300e-02
 -6.2000e-03  1.6800e-02 -3.5700e-02 -2.4900e-02  1.9000e-02 -1.8400e-02
 -5.3700e-02  1.4200e-01  6.0000e-02  2.2600e-02 -3.8000e-03 -6.7500e-02
 -3.6000e-03 -8.0000e-03  5.7000e-02  2.0800e-02  2.2300e-02 -2.5600e-02
 -1.5300e-02  2.2000e-03 -4.8200e-02  1.3100e-02 -6.0160e-01 -8.8000e-03
  1.0600e-02  2.2900e-02  3.3600e-02  7.1000e-03  8.8700e-02  2.3700e-02
 -2.9000e-02 -4.0500e-02 -1.2500e-02  1.4700e-02  4.7500e-02  6.4700e-02
  4.7400e-02  1.9900e-02  4.0800e-02  3.2200e-02  3.6000e-03  3.5000e-02
 -7.2300e-02 -3.0500e-02  1.8400e-02 -

In [81]:
print (en_model['car'])

[-1.600e-02 -3.000e-04 -1.684e-01  8.990e-02 -2.000e-02 -9.300e-03
  4.820e-02 -3.080e-02 -4.510e-02  6.000e-04  1.680e-01  9.650e-02
  3.061e-01 -4.110e-02  2.960e-02 -4.630e-02  3.250e-02 -7.030e-02
  2.220e-02 -1.404e-01 -2.638e-01 -1.340e-02  1.277e-01  1.227e-01
  1.803e-01 -1.920e-02  3.530e-02  1.214e-01  1.509e-01 -8.610e-02
  9.760e-02 -2.550e-02 -2.760e-02 -1.556e-01 -7.390e-02  5.430e-02
 -6.700e-02 -3.000e-03  1.515e-01  6.080e-02  3.300e-02  7.470e-02
  9.000e-04  5.500e-02  4.800e-03 -1.320e-02 -2.620e-02 -1.804e-01
  8.050e-02  4.640e-02 -1.590e-02 -3.020e-02 -6.785e-01  1.632e-01
  1.030e-02  6.550e-02 -8.430e-02  2.270e-02  3.350e-02 -3.560e-02
 -6.380e-02 -1.111e-01 -1.700e-03  9.780e-02  5.650e-02 -3.520e-02
  3.950e-02  1.867e-01  7.900e-02 -1.234e-01  1.860e-02  8.900e-02
  1.631e-01  7.830e-02  5.610e-02  1.447e-01 -2.510e-02  1.376e-01
 -7.900e-03 -2.390e-02  2.180e-02  1.494e-01 -1.910e-02 -2.479e-01
 -4.990e-02  5.160e-02 -1.298e-01 -6.480e-02  2.738e-01  7.800

In [82]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import re

use the rotten tomtoes movie review data https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews/data

In [83]:
df = pd.read_csv('/Users/jianwenliu/Documents/IndianaUniversity/courses/nlp/nlphw/train.tsv',sep='\t')
df.head(10)

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
5,6,1,of escapades demonstrating the adage that what...,2
6,7,1,of,2
7,8,1,escapades demonstrating the adage that what is...,2
8,9,1,escapades,2
9,10,1,demonstrating the adage that what is good for ...,2


In [84]:
df.Sentiment[df['Sentiment']<3]=0
df.Sentiment[df['Sentiment']>=3]=1

df.head(10)

/Users/jianwenliu/anaconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/jianwenliu/anaconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,0
1,2,1,A series of escapades demonstrating the adage ...,0
2,3,1,A series,0
3,4,1,A,0
4,5,1,series,0
5,6,1,of escapades demonstrating the adage that what...,0
6,7,1,of,0
7,8,1,escapades demonstrating the adage that what is...,0
8,9,1,escapades,0
9,10,1,demonstrating the adage that what is good for ...,0


In [85]:
df = df.sample(frac=0.1)

In [86]:
df = df.drop(['PhraseId', 'SentenceId'], axis=1)
df.head()
y_all = df.values[:,-1].reshape(-1,1)

In [87]:
def clean(text):
    return re.compile(u'[^a-zA-Z0-9]').sub(' ',text)   
df['Phrase'] = df['Phrase'].apply(lambda x: clean(x))

In [88]:
def norm_vec(df):
    comm_vec = []
    for com in df.Phrase.values:
        tokens = gensim.utils.tokenize(com,deacc=True, errors='strict', to_lower=True)
        tokens = filter(lambda x: x in en_model.vocab, tokens)
        comm_row = []
        for token in tokens:
            token_vec = np.median(en_model[token])
            comm_row.append(token_vec)
        comm_vec.append(comm_row)
    return comm_vec

In [89]:
comm_vec=norm_vec(df)
comm_vec_tri = [comm[:200] for comm in comm_vec]
import gc
del comm_vec

In [90]:
def zero_pad(A):
    i = 0
    zero_vec = np.zeros(1)
    for comm in A:
        for k in range(0,200-len(comm)):
            comm.insert(0,zero_vec)
        A[i] = comm
        i += 1
    return A

In [91]:
comm_vec_tri=zero_pad(comm_vec_tri)
X = np.array(comm_vec_tri)
import gc
del comm_vec_tri
gc.collect()

8

In [92]:
print(X.shape)
print(y_all.shape)

(15606, 200)
(15606, 1)


In [93]:
X_train, X_test, y_train, y_test = train_test_split(X, y_all, test_size=0.2)

In [94]:
y_train
y_train=y_train.astype('int')
y_test=y_test.astype('int')

In [95]:
from sklearn.linear_model import LogisticRegression

# train the model
clf = LogisticRegression()
clf.fit(X_train, y_train)

/Users/jianwenliu/anaconda3/envs/nlp/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jianwenliu/anaconda3/envs/nlp/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [96]:
from sklearn.metrics import accuracy_score

In [97]:
y_pre = clf.predict(X_test)
acc = accuracy_score(y_test, y_pre)
print("Accuracy is ", acc)

Accuracy is  0.7319026265214607


# POS vector

## use NLP features POS tag for ML

In [60]:
import spacy
from spacy import displacy

nlp = spacy.load('en')

same loading data and prepare process before

In [61]:
df = pd.read_csv('/Users/jianwenliu/Documents/IndianaUniversity/courses/nlp/nlphw/train.tsv',sep='\t')
df.head(10)

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
5,6,1,of escapades demonstrating the adage that what...,2
6,7,1,of,2
7,8,1,escapades demonstrating the adage that what is...,2
8,9,1,escapades,2
9,10,1,demonstrating the adage that what is good for ...,2


In [62]:
df = df.sample(frac=0.1)

df.Sentiment[df['Sentiment']<3]=0
df.Sentiment[df['Sentiment']>=3]=1

df.head(10)

/Users/jianwenliu/anaconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/jianwenliu/anaconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,PhraseId,SentenceId,Phrase,Sentiment
90309,90310,4701,utilizes the idea of making Kahlo 's art a liv...,1
128995,128996,6937,without sacrificing the integrity of the opera,1
62210,62211,3145,This would-be ` James Bond,0
23,24,1,for the goose,0
55806,55807,2791,New York 's finest and a nicely understated ex...,1
26256,26257,1203,is so pedestrian,0
120582,120583,6448,und drang,0
44899,44900,2178,"In its understanding , often funny way",1
114511,114512,6092,Despite the predictable parent vs. child comin...,0
79490,79491,4093,the complicated love triangle that develops be...,0


In [63]:
df = df.drop(['PhraseId', 'SentenceId'], axis=1)

In [64]:
import re
def clean(txt):
    return re.compile(u'[^a-zA-Z0-9]').sub(' ',txt)   
df['Phrase'] = df['Phrase'].apply(lambda x: clean(x))

In [65]:
df.head(10)

,Phrase,Sentiment
90309,utilizes the idea of making Kahlo s art a liv...,1
128995,without sacrificing the integrity of the opera,1
62210,This would be James Bond,0
23,for the goose,0
55806,New York s finest and a nicely understated ex...,1
26256,is so pedestrian,0
120582,und drang,0
44899,In its understanding often funny way,1
114511,Despite the predictable parent vs child comin...,0
79490,the complicated love triangle that develops be...,0


In [66]:
X = df.Phrase.values

In [67]:

X_train, X_test, y_train, y_test = train_test_split(X, df.Sentiment.values, test_size=0.2, random_state=42)

we use pos tags for NLP feature.
First, build the vocab with whole X data.

In [68]:
limit=1000

#build the whole vocab

def get_pos_vec(gen):

    
    #use pos tags for NLP feature
    pos_voc = dict()
    for doc in gen:
        for token in doc:
            pos_voc[token.tag_] = 1

    #count on pos
    ind = 0
    for key, val in pos_voc.items():
        ind += 1
        pos_voc[key] = ind

    return pos_voc

def gen(X,limit):
    for text in X[:limit]:
        yield nlp(text)

gen=gen(X,limit)

pos_voc=get_pos_vec(gen)
print (pos_voc)

{'VBZ': 1, 'DT': 2, 'NN': 3, 'IN': 4, 'VBG': 5, 'NNP': 6, '': 7, '_SP': 8, 'RB': 9, 'PRP$': 10, 'JJ': 11, 'MD': 12, 'VB': 13, 'POS': 14, 'JJS': 15, 'CC': 16, 'VBN': 17, 'WDT': 18, 'CD': 19, 'NNS': 20, 'PDT': 21, 'VBP': 22, 'JJR': 23, 'TO': 24, 'PRP': 25, 'VBD': 26, 'XX': 27, 'EX': 28, 'RP': 29, 'WP': 30, 'WRB': 31, 'UH': 32, 'RBR': 33, 'RBS': 34, 'NNPS': 35, 'FW': 36, 'LS': 37, 'WP$': 38, 'AFX': 39}


In [69]:
# for training data,build the list
#limit to the max length

def X_gen(x):
    for text in x:
        doc = nlp(text)
        yield [pos_voc.get(token.tag_,999) for token in doc]

train_gen = X_gen(X_train)
test_gen = X_gen(X_test)

length = 0
def get_vec(gen,length):
    # append to the dict
    x_vec = list()

    for vec in gen:
        x_vec.append(vec)
        vec_len = len(vec)
        if vec_len > length:
            length = vec_len
    return x_vec,length

In [70]:
#get the vec
train_vec,length_train=get_vec(train_gen,length)
test_vec,length_test=get_vec(test_gen,length)

In [71]:
#padding
#use the maxlength for padding
length=max(length_train,length_test)

def padded_vec(X,length):
    i=0
    for vector in X:
        X[i] = [0] * (length - len(vector)) + vector
        i += 1
    return X

#get the padded train and test vec
train_vec=padded_vec(train_vec,length)
test_vec=padded_vec(test_vec,length)
X_train_pos = np.array(train_vec)
X_test_pos = np.array(test_vec)



In [72]:
#check the shape
X_train_pos.shape

(12484, 54)

In [73]:
X_test_pos.shape

(3122, 54)

In [74]:
#reshape
X_train_f=X_train_pos.reshape(X_train_pos.shape[0], length)

In [75]:
X_test_f=X_test_pos.reshape(X_test_pos.shape[0], length)

In [76]:
#apply logistic regression
from sklearn.linear_model import LogisticRegression

# train
clf = LogisticRegression()
clf.fit(X_train_f, y_train)

/Users/jianwenliu/anaconda3/envs/nlp/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [77]:
from sklearn.metrics import accuracy_score

## Result

In [78]:
#check the accuracy
y_pre = clf.predict(X_test_f)
acc = accuracy_score(y_test, y_pre)
print("Accuracy is ", acc)

Accuracy is  0.7229340166559898


## Discussion

We can find ou that using POS as NLP features can succesfully implementing the sentiment analysis job. With the POS features, the accuracy will reach 0.7229340166559898. However, it is still lower than the result used by well pre trained Spacy Normalized vectors and the Fasttext vector.
This maybe be improved by 1)change to other NLP features like dependency parsing.2) use NLP features conbined with the original BOW features.